In [79]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer
import numpy as np
import evaluate

In [80]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

c:\Users\lwolf\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at C:\Users\lwolf/.cache\huggingface\hub\models--google-bert--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden

In [81]:
from datasets import load_dataset

ds = load_dataset("dalopeza98/isear-cleaned-dataset")

In [82]:
num_to_emotion = {
    0:"anger",
    1:"disgust",
    2:"fear",
    3:"guilt",
    4:"joy",
    5:"sadness",
    6:"shame"
}

In [83]:
n = 74
text = ds["train"][n]["text"]
label = num_to_emotion[ds["train"][n]["label"]]
print(text)
print()
print(label)

I was drinking a glass of wine and eating something when there á
suddenly appeared pictures about the famine in some parts of á
Africa

shame


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = ds["train"].map(tokenize_function)
tokenized_test = ds["test"].map(tokenize_function)

Map:   0%|          | 0/6027 [00:00<?, ? examples/s]

In [85]:
len(tokenized_train["text"])

6027

In [86]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6027
})

In [87]:
# training_args = TrainingArguments(output_dir="test_trainer", per_device_train_batch_size=11, per_device_eval_batch_size=1)
training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [88]:
metric = evaluate.load("accuracy")

In [89]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [90]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 7.26 GiB already allocated; 0 bytes free; 7.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: text. If text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6027
  Num Epochs = 3
  Instantaneous batch size per device = 11
  Total train batch size (w. parallel, distributed & accumulation) = 11
  Gradient Accumulation steps = 1
  Total optimization steps = 1644
  Number of trainable parameters = 109514298


  0%|          | 0/1644 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 132.00 MiB (GPU 0; 8.00 GiB total capacity; 7.16 GiB already allocated; 0 bytes free; 7.19 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF